In [1]:
!pip install evaluate seqeval transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=adaea65ba942333ea5629e8059491d8e11b10cc2e9bf9b66a0f4b354dd77a2f5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.

In [2]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, AutoModelForTokenClassification
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn.metrics import classification_report
from evaluate import load

# Print a message to confirm successful imports
print("Libraries imported successfully.")

Libraries imported successfully.


In [3]:
train_file = "/content/train.tsv"
test_file = "/content/test.tsv"
print(f"Train file: {train_file}")
print(f"Test file: {test_file}")

Train file: /content/train.tsv
Test file: /content/test.tsv


In [4]:
def read_tsv(file_path):
    sentences, labels = [], []
    sentence, label = [], []

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line separates sentences
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
            else:
                token, tag = line.split("\t")
                sentence.append(token)
                label.append(tag)

        # Add the last sentence if the file does not end with a blank line
        if sentence:
            sentences.append(sentence)
            labels.append(label)

    return sentences, labels

# Load datasets
train_sentences, train_labels = read_tsv(train_file)
test_sentences, test_labels = read_tsv(test_file)

# Print a sample for verification
print("Sample Sentence:", train_sentences[0])
print("Sample Labels:", train_labels[0])

Sample Sentence: ['IL-2']
Sample Labels: ['B-DNA']


In [5]:
unique_labels = set(tag for label_seq in train_labels + test_labels for tag in label_seq)
label_to_id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}

print("Label to ID mapping:", label_to_id)

Label to ID mapping: {'B-DNA': 0, 'B-RNA': 1, 'B-cell_line': 2, 'B-cell_type': 3, 'B-protein': 4, 'E-DNA': 5, 'E-RNA': 6, 'E-cell_line': 7, 'E-cell_type': 8, 'E-protein': 9, 'I-DNA': 10, 'I-RNA': 11, 'I-cell_line': 12, 'I-cell_type': 13, 'I-protein': 14, 'O': 15, 'S-DNA': 16, 'S-RNA': 17, 'S-cell_line': 18, 'S-cell_type': 19, 'S-protein': 20}


In [6]:
from datasets import Dataset

def preprocess_data(sentences, labels, label_to_id):
    data = {"tokens": [], "labels": []}

    for sentence, label_seq in zip(sentences, labels):
        # Convert IOBES labels to numeric IDs
        label_ids = [label_to_id[label] for label in label_seq]
        data["tokens"].append(sentence)
        data["labels"].append(label_ids)

    return Dataset.from_dict(data)

# Preprocess train and test datasets
train_data = preprocess_data(train_sentences, train_labels, label_to_id)
test_data = preprocess_data(test_sentences, test_labels, label_to_id)

# Print dataset info
print(train_data)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 18534
})


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
    )

    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(labels[word_idx] if word_idx < len(labels) else -100)
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx

        all_labels.append(aligned_labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

# Initialize tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize datasets
tokenized_train_data = train_data.map(tokenize_and_align_labels, batched=True)
tokenized_test_data = test_data.map(tokenize_and_align_labels, batched=True)

print(tokenized_train_data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Map:   0%|          | 0/18534 [00:00<?, ? examples/s]

Map:   0%|          | 0/4243 [00:00<?, ? examples/s]

{'tokens': ['IL-2'], 'labels': [-100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 14596, 118, 123, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
metric = load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) in predictions and labels
    true_predictions = [
        [id_to_label[pred] for pred, label in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[label] for pred, label in zip(prediction, label_seq) if label != -100]
        for prediction, label_seq in zip(predictions, labels)
    ]

    # Compute metrics
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_to_id)
)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Import the Trainer class
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Handles evaluation at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,  # Keeps only the last 2 checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-8ef8dd2c3ef2>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.212200,0.224854,0.657680,0.789259,0.717487,0.928524
2,0.152600,0.235002,0.672427,0.779355,0.721953,0.931331
3,0.107000,0.271871,0.671059,0.779025,0.721023,0.930141
4,0.069200,0.299259,0.677782,0.769451,0.720713,0.930974
5,0.044200,0.339915,0.678884,0.768460,0.720900,0.930726


TrainOutput(global_step=11585, training_loss=0.12481797763279506, metrics={'train_runtime': 2775.2569, 'train_samples_per_second': 33.392, 'train_steps_per_second': 4.174, 'total_flos': 6054632484733440.0, 'train_loss': 0.12481797763279506, 'epoch': 5.0})

In [10]:
from sklearn.metrics import classification_report

# Evaluate model
predictions, labels, _ = trainer.predict(tokenized_test_data)
predictions = np.argmax(predictions, axis=2)

# Align predictions and labels
true_predictions = [
    [id_to_label[pred] for pred, label in zip(prediction, label_seq) if label != -100]
    for prediction, label_seq in zip(predictions, labels)
]
true_labels = [
    [id_to_label[label] for pred, label in zip(prediction, label_seq) if label != -100]
    for prediction, label_seq in zip(predictions, labels)
]

# Generate classification report
report = classification_report(
    [label for sublist in true_labels for label in sublist],
    [label for sublist in true_predictions for label in sublist],
    zero_division=0
)
print(report)

              precision    recall  f1-score   support

       B-DNA       0.75      0.82      0.78       850
       B-RNA       0.77      0.77      0.77       102
 B-cell_line       0.60      0.75      0.66       425
 B-cell_type       0.82      0.72      0.77      1455
   B-protein       0.74      0.80      0.77      2351
       E-DNA       0.81      0.89      0.85       850
       E-RNA       0.84      0.90      0.87       102
 E-cell_line       0.64      0.81      0.72       425
 E-cell_type       0.88      0.79      0.83      1455
   E-protein       0.77      0.83      0.80      2353
       I-DNA       0.78      0.81      0.80       939
       I-RNA       0.72      0.67      0.70        85
 I-cell_line       0.53      0.73      0.62       562
 I-cell_type       0.83      0.74      0.78      1532
   I-protein       0.76      0.76      0.76      2413
           O       0.98      0.96      0.97     81443
       S-DNA       0.45      0.41      0.43       206
       S-RNA       0.27    